<a href="https://colab.research.google.com/github/takatakamanbou/ML/blob/2023/ex02notebookA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML ex02notebookA

<img width=72 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/ML-logo.png"> [この授業のウェブページ](https://www-tlab.math.ryukoku.ac.jp/wiki/?ML/2023)


----
## 回帰のための教師あり学習(1) 平面の当てはめ
----

----
### 準備


以下，コードセルを上から順に実行してながら読んでいってね．

In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

次のセルを実行すると，データを Pandas の DataFrame として読み込みます．
ここでは Pandas/DataFrame についての理解は求めませんので，このコードセルが何をやっているかはわからなくても構いません．

ここで読み込んでいるデータは，

「Pythonで理解する統計解析の基礎」 谷合廣紀，辻 真吾，技術評論社，2018.

に掲載されているものです．以下の GitHub サイトで公開されています．
https://github.com/ghmagazine/python_stat_sample

In [ ]:
# データの読み込み
df = pd.read_csv('https://github.com/ghmagazine/python_stat_sample/raw/master/data/ch12_scores_reg.csv')
df.drop(columns='通学方法', inplace=True)
df

このデータは，とある授業の受講者20名について，「小テスト」と「期末テスト」の点数，および各受講者の「期末テスト」前日の「睡眠時間」という3つのデータを集めたものとなっています（元のデータにはこれ以外に「通学方法」というのもありますが，ここでは省いています）．




----
### 平面当てはめの問題設定



上記のデータを見たときに，「小テスト」と「睡眠時間」が「期末テスト」に影響していると考えるのは自然なことでしょう．「小テスト」と「睡眠時間」の値から「期末テスト」の値を予測する問題を考えてみます．
「ゴリゴリ君」のときは，気温 $x$ からアイス売上数 $y$ を予測するために $y = ax+b$ という式を立てましたが，データの場合は $x$ に相当する数値が2つあります．

一般化して考えると，このような問題は，$x_1, x_2, \ldots, x_D$ という $D$ 個の値（いまは $D = 2$）から一つの値 $y$ を予測する問題ということができます．このとき，$x_1, \ldots , x_D$ から $y$ を予測する式として単純なのは，

$$
y = w_0 + w_1x_1 + w_2x_2 + \cdots + w_Dx_D \qquad (1)
$$

という形のものです．入力が1つの場合は直線の式でパラメータが2つだったのに対して，この式ではパラメータが $w_0, w_1, \ldots , w_D$ の $(D+1)$ 個あります．

式(1)は，$D=1$ のときは2次元平面上の直線，$D=2$ のときは3次元空間中の平面となり，一般の $D$ については，$(D+1)$次元空間中の $D$ 次元超平面となります．
以下の図は，2次元のデータに対する平面当てはめの例です．個々の青い点は，$(x_1, x_2, そのときの y の正解)$ を表し，赤いメッシュがこれらのデータに当てはめた平面を表しています．



<img src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/planefitting.png" width="50%">


入力が1つ（1次元）の直線当てはめの問題では，最小二乗法によって最適な（二乗誤差の和が最小となる）パラメータを求めました．入力がD個（$D$次元）の場合も同様に，最小二乗法によってデータに当てはまる（超）平面のパラメータを求められます．


以下，最小二乗法による平面当てはめの問題設定を整理して説明します．ただし，式の見通しをよくするために，$1, x_1, x_2, \ldots, x_D$ という $(D+1)$ 個の値をならべた $(D+1)$ 次元ベクトルを 
$$
\mathbf{x} = (1, x_1, x_2, \ldots, x_D)
$$
と表し（先頭に$1$があることに注意），$(D+1)$個のパラメータをならべた $(D+1)$ 次元ベクトルを 
$$
\mathbf{w} = (w_0, w_1, w_2, \ldots, w_D)
$$
と表すことにします．こうしておくと，式$(1)$を
$$
y = f(\mathbf{x}) = \mathbf{w}\cdot\mathbf{x} \qquad (2)
$$
と簡単な形で書けます（注）．

※注: 記号$\cdot$は普通のベクトル同士の内積です．$\mathbf{w}\cdot\mathbf{x} = w_0\times1 + w_1\times x_1+\ldots+w_D\times x_D$.



**［最小二乗法による平面当てはめの問題設定］**

$D$ 個の値 $x_1, x_2, \ldots, x_D$ から $y$ の値が決まるようなデータがあり，それらの間の関係として式(2)が仮定できるとする．$N$個のデータ
$$
(\mathbf{x}_1, y_1), (\mathbf{x}_2, y_2),\ldots , (\mathbf{x}_N, y_N)
$$
が与えられる．ただし，$\mathbf{x}_n$は上述のように$1$と$D$個の値をならべた$(D+1)$次元ベクトルであり，$y_n$ はこのデータに対する $y$ の値の正解である（$n=1,2,\ldots, N$）．

このとき，$f(\mathbf{x}_n)$ とその正解の値 $y_n$ との間の二乗誤差の和
$$
\sum_{n=1}^{N}(y_n - f(\mathbf{x}_n))^2 =\sum_{n=1}^{N}(y_n - \mathbf{w}\cdot\mathbf{x}_n)^2 \qquad (3)
$$
を最小にするパラメータ $\mathbf{w}$ を求めたい．

----
### 平面当てはめの最小二乗法の解


まず，ベクトル $\mathbf{x}_n$ を行ベクトル（要素が行方向に並んだベクトル）とみなして，それらを列方向に並べた行列を $X$ とします．

$$
X = \begin{pmatrix}
\mathbf{x}_1\\
\mathbf{x}_2\\
\vdots\\
\mathbf{x}_N\\
\end{pmatrix}
=
\begin{pmatrix}
1 & x_{1,1} & x_{1,2} & \cdots & x_{1,D}\\
1 & x_{2,1} & x_{2,2} & \cdots & x_{2,D}\\
& & \vdots\\
1 & x_{N,1} & x_{N,2} & \cdots & x_{N,D}\\
\end{pmatrix}
$$

$X$ は $N\times(D+1)$ の行列です．次に，出力の正解の値 $y_1, y_2, \ldots, y_N$ を列方向に並べた行列を $Y$ とします．


$$
Y = \begin{pmatrix}
y_1\\ y_2 \\ \vdots \\ y_N
\end{pmatrix}
$$

$Y$ は $N\times 1$ の行列です．
このとき，正規方程式は次式のようになります（正規方程式がこのようになることを示すのは後の節に回します）．

$$
X^{\top}X
\begin{pmatrix}
w_0\\ w_1\\ \vdots \\ w_D
\end{pmatrix}
= X^{\top}Y \qquad (4)
$$

この式の形からわかるように，正規方程式は $(D+1)$元の連立方程式です．

したがって，与えられた学習データから行列 $X, Y$ を作り，$X^{\top}X$ と $X^{\top}Y$ を計算して式$(4)$を解けば，二乗誤差の和を最小とするパラメータ $w_0, w_1, \ldots, w_{D}$ が求まります．


----
### 「小テスト」と「睡眠時間」から「期末テスト」を予測するモデルを作る

実際に平面当てはめの実験をやってみましょう．


In [ ]:
# N x (D+1) 行列 X をつくる
X_raw = df.loc[:, ['小テスト', '睡眠時間']].to_numpy()
X = np.vstack([np.ones(len(X_raw)), X_raw.T]).T
print(X)
print(X.shape)

In [ ]:
# 正解の値をならべたベクトル Y をつくる
Y = df['期末テスト'].to_numpy()
print(Y)
print(Y.shape)

In [ ]:
# 正規方程式を解く
XTX = X.T @ X  # 正規方程式の左辺の(D+1)x(D+1)の行列
XTY = X.T @ Y    # 正規方程式の右辺の(D+1)x1の行列
w = np.linalg.solve(XTX, XTY) # 連立方程式を解く
print(f'{len(w)}個のパラメータの値は')
print(w)

学習データに対して二乗誤差の和を最小にする3つのパラメータ $w_0, w_1, w_2$ の値が得られました．
このパラメータで予測がどの程度できるのか試してみましょう．

In [ ]:
# 予測値の計算
Yt = X @ w

print(' 小テスト 睡眠 期末  予測値')
for n in range(len(X)):
    print(f'{n:2d}  {X[n, 1]}  {X[n, 2]}  {Y[n]}  {Yt[n]:.2f}')

それっぽい値を出しているのもあれば，大きく外しているのもありますね．

★ やってみよう

次のセルを実行すると何が得られるのか，上のセルの内容および実行結果と比較して考えなさい．
それをふまえて，小テストが5点で睡眠時間が8時間のひとと，小テストが8点で睡眠時間が5時間のひとの期末テストの得点の予測値を求めなさい．結果を紙媒体にメモしておきなさい．

In [ ]:
XX = np.array([
    [1, 4.2, 7.2],
    [1, 7.2, 7.9],
    ])
Yt = XX @ w
for n in range(len(XX)):
    print(f'{n:2d}  {XX[n, 1]}  {XX[n, 2]}  {Yt[n]:.2f}')

----
### ［補足］ 回帰分析？重回帰分析？

この授業で登場している直線当てはめおよび平面当てはめの方法は，「教師あり学習」の「回帰」のための方法として最も簡単なものです．これらの方法は，データ分析や統計学の分野では，**回帰分析**，**重回帰分析**という名前で知られています（注）．

※注: 変数（被説明変数）が1つなのが（単）回帰，2つ以上なのが重回帰．区別せずどちらも回帰分析ということもあります．

----
### ［発展］平面当てはめの最小二乗法の解の導出

この節では，平面当てはめの最小二乗法の解が式$(4)$の連立方程式の解として求まることを示します．
この節の内容は発展的な話題であり，この授業で必ず修得すべきものとはしません．興味・時間の余裕のあるひとは式を追いかけたり式変形を自分でやってみたりしてください．

(step1) 誤差関数 $E(\mathbf{w})$ を次のように定義します．
$$
E(\mathbf{w}) = \frac{1}{2}\sum_{n=1}^N (y_n - \mathbf{w}\cdot\mathbf{x}_n)^2
$$
ただし，$\mathbf{w}$ は $w_0, w_1, \ldots, w_D$ を並べた $(D+1)$ 次元ベクトルです．

(step2) $E(\mathbf{w})$ が最小となるパラメータを求めるために，これを $w_d$ ($d = 0, 1, 2, \ldots, D$) のそれぞれで偏微分したものを計算します．

$$
\begin{aligned}
\frac{\partial E(\mathbf{w})}{\partial w_d} &= \sum_{n=1}^N (y_n - \mathbf{w}\cdot\mathbf{x}_n) \frac{\partial}{\partial w_d}(y_n - \mathbf{w}\cdot\mathbf{x}_n)   \\
&= \sum_{n=1}^N (y_n - \mathbf{w}\cdot\mathbf{x}_n)\left( 0 -\frac{\partial}{\partial w_d}(w_0+w_1x_{n,1}+\cdots + w_Dx_{n,D})\right) \\
&= \sum_{n=1}^N (y_n - \mathbf{w}\cdot\mathbf{x}_n)(-x_{n,d})\qquad (d = 0, 1, 2, \ldots, D)
\end{aligned}
$$

$\mathbf{x}_n$ も $\mathbf{w}$ も要素は添字の番号が $0$ のものから $D$ のものまである（$(D+1)$個ある）ことに注意してください．さらに，$\mathbf{x}_n$ の $0$ 番目の要素 $x_{n,0}$ は常に $1$ です．


(step3) 
$\frac{\partial E(\mathbf{w})}{\partial w_d} = 0$ とすると，次の式が得られます．

$$
\begin{pmatrix}
\sum x_{n,0}\mathbf{w}\cdot\mathbf{x}_n \\
\sum x_{n,1}\mathbf{w}\cdot\mathbf{x}_n \\
\vdots \\
\sum x_{n,D}\mathbf{w}\cdot\mathbf{x}_n
\end{pmatrix} = 
\begin{pmatrix}
\sum x_{n,0}y_n \\ \sum x_{n,1}y_n\\ \vdots \\ \sum x_{n,D}y_n
\end{pmatrix}
$$

これは，$(D+1)$ 個の未知数 $w_0, w_1, \ldots, w_D$ に対して $(D+1)$ 個の式から成る連立方程式です．これを整理して，式$(4)$を導出します．

この式の左辺の $d$ 番目（$d=0,1,\ldots,D$）の行を取り出して考えると，

$$
\begin{aligned}
\sum_{n=1}^{N} x_{n,d}\mathbf{w}\cdot\mathbf{x}_n &= \sum_{n=1}^{N} (x_{n,d}\mathbf{x}_n)\cdot \mathbf{w} = \left( \sum_{n=1}^{N} x_{n,d}\mathbf{x}_n\right) \cdot\mathbf{w} \\
&= \left( \sum x_{n,d}x_{n,0}, \sum x_{n,d}x_{n,1},  \ldots, \sum x_{n,d}x_{n,D} \right) \cdot \mathbf{w}
\end{aligned}
$$

と書けるので，この式の左辺は

$$
\begin{pmatrix}
\sum x_{n,0}x_{n,0} & \sum x_{n,0}x_{n,1} & \cdots & \sum x_{n,0}x_{n,D} \\
\sum x_{n,1}x_{n,0} & \sum x_{n,1}x_{n,1} & \cdots & \sum x_{n,1}x_{n,D} \\
\vdots & \vdots & & \vdots\\
\sum x_{n,D}x_{n,0} & \sum x_{n,D}x_{n,1} & \cdots & \sum x_{n,D}x_{n,D} \\
\end{pmatrix}
\begin{pmatrix}
w_0\\ w_1\\ \vdots \\ w_D
\end{pmatrix}
= \left(\sum_{n=1}^{N} \mathbf{x}_n^{\top} \mathbf{x}_n \right)
\begin{pmatrix}
w_0\\ w_1\\ \vdots \\ w_D
\end{pmatrix}
= X^{\top}X
\begin{pmatrix}
w_0\\ w_1\\ \vdots \\ w_D
\end{pmatrix}
$$

となります．

右辺の方も同様にして $X^{\top}Y$ と等しいことがわかります（そっちの方が簡単です．自分で導出してみてね）．